## Testing out the environment

In [70]:
import langchain
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
from pprint import pprint

In [2]:
load_dotenv()  # Load environment variables from a .env file if present

True

In [3]:
langchain.__version__

'0.3.27'

In [4]:
gemini_flash_model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [26]:
model = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
)

In [27]:
model.invoke("what is the capital of india").content

'The capital of India is **New\u202fDelhi**.'

In [7]:
gemini_flash_model.invoke("what is the capital of india").content

'The capital of India is **New Delhi**.'

### Embedding model

In [ ]:
embedding_model = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")
embeddings = embedding_model.embed_query(text="What's our Q1 revenue?", output_dimensionality=10)

[-0.03572908416390419,
 0.014558478258550167,
 0.011592254973948002,
 -0.08969993889331818,
 -0.009068180806934834,
 0.013664662837982178,
 0.011340967379510403,
 -0.005701108369976282,
 -0.027033332735300064,
 3.775993536692113e-05]

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings

In [19]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"},
)
embeddings = embedding_model.embed_query(text="What's our Q1 revenue?")

In [22]:
len(embeddings)

384

## Langchain Prompts

### Basic Prompt template

In [61]:
from langchain_core.prompts import PromptTemplate, load_prompt

In [62]:
template = PromptTemplate(
    template="""
    You are a helpful assistant that can generate a short report on the topic: {paper_input}
    The report should be in the style of {style_input} and the length should be {length_input}
    """,
    input_variables=["paper_input", "style_input", "length_input"]
)

template.save("./prompt_templates/template.json")

In [63]:
template = load_prompt("./prompt_templates/template.json")

In [64]:
prompt = template.format(
        paper_input="Attention is all you need", style_input="Beginner-Friendly", length_input="Short (1-2 paragraphs)"
    )

In [67]:
print (prompt)

### Messages


    You are a helpful assistant that can generate a short report on the topic: Attention is all you need
    The report should be in the style of Beginner-Friendly and the length should be Short (1-2 paragraphs)
    


### Messages

In [49]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [35]:
messages = [
    SystemMessage(content="You are a helpful assistant that can answer questions and help with tasks."),
    HumanMessage(content="What is the capital of France?"),
]

result = model.invoke(messages)

result.content

'The capital of France is **Paris**.'

### Dynammic list of messages

In [39]:
chat_template = ChatPromptTemplate(messages=[
    SystemMessage(content="You are a helpful assistant who is an expert in the domain: {domain}"),
    HumanMessage(content="Explain the topic in simple terms: {topic}"),
])

In [42]:
prompt = chat_template.invoke({"domain": "AI", "topic": "Self Attention"})

print(prompt)

messages=[SystemMessage(content='You are a helpful assistant who is an expert in the domain: {domain}', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain the topic in simple terms: {topic}', additional_kwargs={}, response_metadata={})]


### The above does not work

In [47]:
chat_template = ChatPromptTemplate(messages=[
    ("system", "You are a helpful assistant who is an expert in the domain: {domain}"),
    ("human", "Explain the topic in simple terms in 3-5 sentences: {topic}"),
])
prompt = chat_template.invoke({"domain": "AI", "topic": "Self Attention"})

print(prompt)

messages=[SystemMessage(content='You are a helpful assistant who is an expert in the domain: AI', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain the topic in simple terms in 3-5 sentences: Self Attention', additional_kwargs={}, response_metadata={})]


In [48]:
result = model.invoke(prompt)
print(result.content)


Self‑attention is a way for a model to look at all the words in a sentence at once and decide how much each word should influence every other word.  
For each word, the model creates three vectors—query, key, and value—then compares the query of one word with the keys of all words to get a “similarity score.”  
These scores are turned into weights (via a softmax) that say how much attention each word should give to the others, and the weighted sum of the value vectors gives the new representation for that word.  
Because every word can attend to every other word, the model captures long‑range relationships and context without needing to process the sentence sequentially.  
This mechanism is the core of transformer models, enabling them to understand and generate language efficiently.


### Message Placeholder

In [72]:
history_messages = [
    HumanMessage(content="I want to request a refund for my order #12345."),
    AIMessage(content="Your refund request for order #12345 has been initiated. It will be processed in 3-5 business days.")
]

In [75]:
history_messages[0].content


'I want to request a refund for my order #12345.'

In [71]:
chat_template = ChatPromptTemplate(messages=[
    ("system", "You are a helpful assistant that can answer questions and help with tasks."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{query}"),
])

prompt = chat_template.invoke({"query": "how many day again?", "chat_history": history_messages})

pprint(prompt)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant that can answer questions and help with tasks.', additional_kwargs={}, response_metadata={}), HumanMessage(content='I want to request a refund for my order #12345.', additional_kwargs={}, response_metadata={}), AIMessage(content='Your refund request for order #12345 has been initiated. It will be processed in 3-5 business days.', additional_kwargs={}, response_metadata={}), HumanMessage(content='how many day again?', additional_kwargs={}, response_metadata={})])


In [57]:
response = model.invoke(prompt)
print(response.content)

It will take **3–5 business days** to complete the refund.  
That means the processing time is counted only on weekdays (Monday‑Friday), excluding public holidays. If you have any more questions, just let me know!


## Structured Outputs

### Typed Dict

In [84]:
from typing import TypedDict, Annotated, Optional, Literal

class Person(TypedDict):
    name: str
    age: int
    email: str

person = Person(name="John", age=30, email="john@example.com")

In [ ]:
person = Person(name=123, age=30, email="john@example.com") # this is wrong according to the type def, but it will not raise an error

In [78]:
# create a dummy review for a mobile phone in plain text
review = """
This is a decent phone! I love the camera and the battery life is amazing. Also, the price is reasonable. The display is ok, but some more details are that the ppi is 400 and the screen to body ratio is 80%.
Gaming experience is good, but the phone gets heated up after 1 hour of gaming.
Netwok connectivity is good. Overall, it is a good phone for the price.
"""

# create a review schema

class Review(TypedDict):
    summary: str
    sentiment: str

structured_model = model.with_structured_output(Review)
response = structured_model.invoke(review)
pprint(response)

{'sentiment': 'positive',
 'summary': 'The phone offers great camera performance, long battery life, and '
            'a reasonable price, with a decent display and solid gaming '
            'experience, though it heats up after extended play. Overall, a '
            'solid value.'}


`with_structured_output` with a schema results in a system prompt behind the scenes which results in generating the response which follows the provided shema

### Using Annotations

In [80]:
class Review(TypedDict):
    summary: Annotated[str, "A summary of the review"]
    sentiment: Annotated[str, "The sentiment of the review - posutive, negative or neutral"]

structured_model = model.with_structured_output(Review)
response = structured_model.invoke(review)
pprint(response)



{'sentiment': 'positive',
 'summary': 'The phone offers a solid camera, long battery life, and a '
            'reasonable price. The display is decent with a 400\u202fppi and '
            '80% screen‑to‑body ratio. Gaming is enjoyable but the device '
            'tends to heat after an hour. Network connectivity is reliable, '
            'making it a good overall value.'}


In [81]:
# create a detailed review for iPhone 17 Air
iphone_17_air_review = """
The iPhone 17 Air represents Apple's boldest design departure in years, delivering an incredibly thin profile that feels almost impossibly light in hand. At just 5.5mm thick, this device pushes the boundaries of engineering while maintaining the premium build quality we expect from Apple. The aerospace-grade aluminum frame feels solid despite its minimal thickness, and the new Ceramic Shield front provides excellent protection without adding bulk. The device comes in four stunning colors: Midnight Black, Starlight Silver, Deep Purple, and a new Ocean Blue that shifts subtly in different lighting conditions.

Performance-wise, the iPhone 17 Air doesn't compromise despite its slim form factor. The A18 Bionic chip with its 3nm process delivers exceptional speed and efficiency, handling everything from intensive gaming to professional video editing with ease. The 8GB of unified memory ensures smooth multitasking, and the improved Neural Engine makes AI-powered features incredibly responsive. Battery life is surprisingly robust for such a thin device, easily lasting a full day of heavy usage thanks to the more efficient chip and optimized iOS 18 integration. The new MagSafe wireless charging is faster than ever, reaching 25W speeds that rival many wired chargers.

The camera system is where the iPhone 17 Air truly shines, featuring a revolutionary new periscope telephoto lens that somehow fits within the ultra-thin chassis. The main 48MP sensor captures stunning photos with incredible detail and dynamic range, while the new computational photography features produce professional-quality results in challenging lighting conditions. Night mode has been significantly improved, and the new Action mode for video recording delivers gimbal-like stabilization. The front-facing camera now supports 4K ProRes recording, making it perfect for content creators who demand the highest quality.

However, the pursuit of thinness does come with some trade-offs. The device can get noticeably warm during intensive tasks, and the reduced internal space means no room for a traditional headphone jack or even the Lightning port - it's USB-C only with wireless charging as the primary power source. The speakers, while clear, lack the depth and bass response of thicker iPhone models. Additionally, the ultra-thin design makes the device feel somewhat fragile, and Apple's recommended case adds back much of the thickness that the Air design eliminates. Despite these minor compromises, the iPhone 17 Air succeeds in creating a truly premium, futuristic smartphone experience that feels like a glimpse into the next decade of mobile technology.
"""


In [85]:
class Review(TypedDict):
    key_themes: Annotated[list[str], "The key themes of the review as a list of strings"]
    summary: Annotated[str, "A brief summary of the review - max 100 words"]
    sentiment: Annotated[Literal["pos", "neg", "neu"], "The sentiment of the review"]
    pros: Annotated[Optional[list[str]], "The pros of the review as a list of strings"]
    cons: Annotated[Optional[list[str]], "The cons of the review as a list of strings"]

structured_model = model.with_structured_output(Review)
response = structured_model.invoke(iphone_17_air_review)
pprint(response)

{'cons': ['Device can get noticeably warm during intensive tasks',
          'No headphone jack or Lightning port – USB-C only',
          'Speakers lack depth and bass response',
          'Ultra-thin design feels somewhat fragile',
          "Apple's recommended case adds back much of the thickness that the "
          'Air design eliminates'],
 'key_themes': ['Design and Build',
                'Performance and Efficiency',
                'Camera System',
                'Battery Life and Charging',
                'Trade-offs and Limitations'],
 'pros': ['Incredibly thin profile at 5.5mm',
          'Aerospace-grade aluminum frame',
          'Ceramic Shield front protection',
          'Stunning color options',
          'A18 Bionic chip with 3nm process',
          '8GB unified memory',
          'Improved Neural Engine',
          'Robust battery life',
          '25W MagSafe wireless charging',
          'Revolutionary periscope telephoto lens',
          '48MP main sensor',
 

In [ ]:
# create a detailed review for iPhone 17 Air -- without cons
iphone_17_air_review_without_cons = """
The iPhone 17 Air represents Apple's boldest design departure in years, delivering an incredibly thin profile that feels almost impossibly light in hand. At just 5.5mm thick, this device pushes the boundaries of engineering while maintaining the premium build quality we expect from Apple. The aerospace-grade aluminum frame feels solid despite its minimal thickness, and the new Ceramic Shield front provides excellent protection without adding bulk. The device comes in four stunning colors: Midnight Black, Starlight Silver, Deep Purple, and a new Ocean Blue that shifts subtly in different lighting conditions.

Performance-wise, the iPhone 17 Air doesn't compromise despite its slim form factor. The A18 Bionic chip with its 3nm process delivers exceptional speed and efficiency, handling everything from intensive gaming to professional video editing with ease. The 8GB of unified memory ensures smooth multitasking, and the improved Neural Engine makes AI-powered features incredibly responsive. Battery life is surprisingly robust for such a thin device, easily lasting a full day of heavy usage thanks to the more efficient chip and optimized iOS 18 integration. The new MagSafe wireless charging is faster than ever, reaching 25W speeds that rival many wired chargers.

The camera system is where the iPhone 17 Air truly shines, featuring a revolutionary new periscope telephoto lens that somehow fits within the ultra-thin chassis. The main 48MP sensor captures stunning photos with incredible detail and dynamic range, while the new computational photography features produce professional-quality results in challenging lighting conditions. Night mode has been significantly improved, and the new Action mode for video recording delivers gimbal-like stabilization. The front-facing camera now supports 4K ProRes recording, making it perfect for content creators who demand the highest quality.
"""

structured_model = model.with_structured_output(Review)
response = structured_model.invoke(iphone_17_air_review_without_cons)
pprint(response)



{'cons': [],
 'key_themes': ['Design & Build',
                'Performance & Efficiency',
                'Battery & Charging',
                'Camera & Photography',
                'User Experience'],
 'pros': ['Ultra-thin 5.5mm profile with premium aerospace-grade aluminum '
          'frame',
          'Exceptional performance from A18 Bionic 3nm chip and 8GB RAM',
          'Robust battery life and fast 25W MagSafe charging',
          'Revolutionary periscope telephoto lens and 48MP main sensor',
          'Advanced computational photography and improved night mode',
          'Front camera supports 4K ProRes for creators'],
 'sentiment': 'pos',
 'summary': 'The iPhone\u202f17\u202fAir redefines slimness with a 5.5\u202fmm '
            'chassis that feels surprisingly solid, thanks to aerospace‑grade '
            'aluminum and Ceramic Shield. Powered by the A18 Bionic 3\u202fnm '
            'chip and 8\u202fGB RAM, it delivers gaming‑grade speed, efficient '
            'mul

This works really well -- but there is no data validation - this can be done by Pydantic

### Using Pydantic